
# Defining Dynamic FBA model for a single species




## Import headers



In [1]:
%matplotlib inline
import cobra
import PyDSTool as dst
import pandas as pd
import matplotlib.pyplot as plt
import copy

:RESULTS:
nil:END:




## File Paths for Media spec and FBA model



In [2]:
model = cobra.io.read_sbml_model('./data/Bacteroides_sp_1_1_14.xml')
mediaFluxes = pd.read_csv('./data/VMH_HighFiber.tsv', sep='\t')

:RESULTS:
nil:END:




## Extract exchange reactions into variable definition and ics dictionary



In [3]:
def cleanupname(name):
    """
    The reaction names in the model files 
    don't have brackets or parentheses. I replaced
    those found in the mediaFluxes file.
    """
    name = name.replace('[', '_LPAREN_')
    name = name.replace(']', '_RPAREN_')
    name = name.replace('(', '_LPAREN_')
    name = name.replace(')', '_RPAREN_')
    return name

"""
Initialize dictionaries for defining the ODE variables for a 
single species
"""

VarDef = dict()
ParDef_base = dict()
ICS_base = dict()

solution = model.optimize() # Get fluxes
all_exchanges = model.exchanges
media_derived_components = dict()

exchange_reaction_ids = [rxn.id for rxn in all_exchanges]

AttributeError: 'Model' object has no attribute 'exchanges'


### Extract the reaction names and flux values into a dictionary.



In [1]:
for i, row in mediaFluxes.iterrows():
    N = cleanupname(row.Reaction)
    if N in exchange_reaction_ids:
        media_derived_components[N] = row['Flux Value'] / (24.0*60.0) # Per minute

:RESULTS:
nil:END:




### Get the outfluxes



In [1]:
out_fluxes = []
for k in all_exchanges:
    if k.reversibility and k.id not in media_derived_components.keys():
        out_fluxes.append(k.id)

ParDef_base['mu'] = solution.objective_value/60.0
VarDef['X'] =  'mu * X'
ICS_base['X'] = 1e-1
for r in all_exchanges:
    if r.id in media_derived_components.keys():
        ParDef_base[r.id + '_influx'] = media_derived_components[r.id]
        ParDef_base[r.id + '_consumption'] = solution.fluxes[r.id]
        ICS_base[r.id] = media_derived_components[r.id]
        VarDef[r.id] = r.id + '_influx + ' + r.id + '_consumption' + ' * X'
    if r.id in out_fluxes:
        ParDef_base[r.id + '_outflux'] = solution.fluxes[r.id] 
        VarDef[r.id] = r.id + '_outflux * X'
        ICS_base[r.id] = 0.0

:RESULTS:
nil:END:




## Define PyDSTool Model



In [8]:
TSPAN = 5

ParDef = copy.deepcopy(ParDef_base)
ICS = copy.deepcopy(ICS_base)
def simulateModel(ModelDS, ParDef, ICS, TSPAN, tstart):
    ModelDS.set(ics=ICS ,pars=ParDef,tdata=[tstart, tstart + TSPAN])
    P = ModelDS.compute('test').sample(dt=0.1)
    return(P)
ParDef = copy.deepcopy(ParDef_base)
ICS = copy.deepcopy(ICS_base)
ModelDef =dst.args(name='Bacteroides',
                  varspecs=VarDef,
                  pars=ParDef,
                  ics=ICS,
                  )

ModelDS = dst.Vode_ODEsystem(ModelDef)

P = simulateModel(ModelDS, ParDef, ICS,TSPAN, 0.0)

AssertionError: varspecs attribute must be defined

:RESULTS:
nil:END:




## Create Upperlimit



In [1]:
AllPoints = []
AllPoints.append(P)
OriginalLB = {}
model_copy = copy.deepcopy(model)
T0=TSPAN

for r in model_copy.reactions:
    if r in model_copy.exchanges:
        OriginalLB[r.id] = r.lower_bound

for i in range(0, 5):
    print(T0)
    solution = model_copy.optimize()
    all_exchanges = model_copy.exchanges
    for r in all_exchanges:
        if r.id in media_derived_components.keys():
            model_copy.reactions.get_by_id(r.id).lower_bound = OriginalLB[r.id]*P[r.id][-1]/(0.0005+P[r.id][-1])
            if r.id == 'EX_glu_L_LPAREN_e_RPAREN_':
                print(r.id + ': ' +str(OriginalLB[r.id]) + ' -> ' + str(model_copy.reactions.get_by_id(r.id).lower_bound))
    solution = model_copy.optimize()
 #   print('mu= ' +str(solution.objective_value))
    ParDef['mu'] = solution.objective_value/60.0
    ICS['X'] = P['X'][-1]
    for r in all_exchanges:
        if r.id in media_derived_components.keys():
            ParDef[r.id + '_influx'] = media_derived_components[r.id]
            ParDef[r.id + '_consumption'] = solution.fluxes[r.id]
            ICS[r.id] = P[r.id][-1]
        if r.id in out_fluxes:
            ParDef[r.id + '_outflux'] = solution.fluxes[r.id] 
            ICS[r.id] = P[r.id][-1] # 0.0
    del P
    P = simulateModel(ModelDS, ParDef, ICS,TSPAN, T0)
#    print('EX_glu_L_LPAREN_e_RPAREN_' + str(P['EX_glu_L_LPAREN_e_RPAREN_'][-1]))
    T0 += TSPAN

    AllPoints.append(P)

4 - 4463804b-d1a2-4de0-b6a1-785e6cff0cce

In [5]:
T=[]
 X=[]
for v in VarDef.keys():
        for P in AllPoints:
                # if 'h2o' not in v :
                #         print(v)
                #         T = P['t']
                #         X = P[v]
                #         plt.plot(T,X)
                T = P['t']
                X = P['X']
                plt.plot(T,X)

plt.xlabel('Time (minutes)')
plt.ylabel('gdw')

IndentationError: unexpected indent (<ipython-input-5-0108591d04cd>, line 2)